In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install pyspark
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-26 20:18:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.46MB/s    in 0.2s    

2022-01-26 20:19:00 (4.46 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLAmazonToys").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
toy_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")
# Show DataFrame
toy_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-01-31 00:08:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

In [5]:
#Count the number of records (rows) in the dataset.
print(toy_data_df.count())
toy_data_df=toy_data_df.dropDuplicates()
toy_data_df=toy_data_df.dropna()
print(toy_data_df.count())

4864249
4863497


**Transform the dataset to fit the tables in the schema file.**

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col,to_date

**Review ID Table**

In [7]:
review_id_table_df=toy_data_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
print(review_id_table_df.count())
print(review_id_table_df.distinct().count())
review_id_table_df.show(5)

4863497
4863497
+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100FHBHYPZTQP|   45689333|B00146G71U|     337661758|2015-01-16 00:01:00|
| R1019T4WSRNZR|    4793524|B00262SNBI|     257790875|2014-01-21 00:03:00|
|R10218N1NMX46P|   32005660|B00K0PVVAI|     931670277|2014-01-28 00:09:00|
|R1024Y1W1EQDZ3|    4726506|B00BGZLSJK|     916634683|2013-01-02 00:04:00|
|R1029KDS040YBY|   19752183|B007YD77MK|     993813478|2013-01-18 00:01:00|
|R102BTUCOTG4HN|   24005420|B0027FFPJC|     388549569|2010-01-26 00:12:00|
|R102IMDHQMC5S0|   44902834|B001P5GRYW|     496075093|2009-01-23 00:04:00|
| R102J7VZ9Q8Z1|   25053850|B001USHRYI|     625435419|2009-01-26 00:08:00|
|R102PIFZ2BY5E2|   15694296|B004P13H5U|     173176560|2014-01-16 00:01:00|
|R102REHQU6V176|   45261348|B00160DXNE|     112689753|2015-01-20 00:03:00|
|R102T6WA

**Customer Table**


In [14]:
customer_df=toy_data_df.select(['customer_id'])
customer_df=customer_df.groupBy('customer_id').count()
customer_df = customer_df.withColumnRenamed("count", "customer_count") 
customer_df=customer_df.dropDuplicates()
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4409437|             5|
|   23810272|             1|
|   44015381|             9|
|   22827350|             2|
|    6074522|             7|
|   33326996|             2|
|   10165796|             2|
|   37822161|            11|
|   45802649|            17|
|   27632536|             3|
|   18068011|            18|
|   26354432|             4|
|   40453343|             5|
|   15404449|            13|
|     729725|             8|
|   15508207|            11|
|   21866561|             1|
|   33795243|             1|
|   13913228|             7|
|   42957529|             2|
+-----------+--------------+
only showing top 20 rows



**Product Table**

In [9]:
products_df=toy_data_df.select(['product_id','product_title'])
print(products_df.count())
print(products_df.distinct().count())
products_df=products_df.dropDuplicates()
print(products_df.count())
products_df.show()

4863497
663991
663991
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows



**Vine Table**

In [10]:
vine_table_df=toy_data_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
print(vine_table_df.count())
print(vine_table_df.distinct().count())
vine_table_df.show()

4863497
4863497
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100FHBHYPZTQP|          4|            0|          0|   N|
| R1019T4WSRNZR|          5|            2|          2|   N|
|R10218N1NMX46P|          5|            0|          0|   N|
|R1024Y1W1EQDZ3|          2|            1|          2|   N|
|R1029KDS040YBY|          5|            0|          0|   N|
|R102BTUCOTG4HN|          5|            0|          0|   N|
|R102IMDHQMC5S0|          3|            0|          0|   N|
| R102J7VZ9Q8Z1|          5|            5|          5|   N|
|R102PIFZ2BY5E2|          4|            0|          0|   N|
|R102REHQU6V176|          5|            0|          0|   N|
|R102T6WA4AP028|          1|            7|          7|   N|
|R102U3DV9AUAUR|          5|            3|          3|   N|
|R10373WGOF7NH0|          2|            1|          1|   N|
|R1037MRBABLNXG|        

**Loading to Database**

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/Amazon_Review"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [12]:
review_id_table_df.write.jdbc(url=jdbc_url,table='review_id_table',mode=mode,properties=config)

In [16]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
products_df.write.jdbc(url=jdbc_url,table='products',mode=mode,properties=config)

In [18]:
vine_table_df.write.jdbc(url=jdbc_url,table='vine_table',mode=mode,properties=config)

# New Section